### 사전환경 설정 및 github에서 styleGAN2-ada모델 가져오는 과정

In [ ]:
# 코랩과 구글드라이브를 연동해서 사용하려면 주석을 풀고 이 셀을 실행시키면 됩니다.
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# 이 명령어는 코랩에서만 작동됩니다. 주피터 노트북의 경우 tensorflow 버전을 다른 명령어로 사용하여 바꿔줘야합니다. 
%tensorflow_version 1.x

In [1]:
# 그래픽카드의 정보를 확인할 수 있는 명령어 입니다. 이 명령어 또한 코랩에서만 작동합니다.
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [2]:
import os
if os.path.isdir("/content/drive/My Drive/colab-sg2-ada"):
    %cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
else:
    #install script
    %cd "/content/drive/My Drive/"
    !mkdir colab-sg2-ada
    %cd colab-sg2-ada
    !git clone https://github.com/NVlabs/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

/Users/parkheondo/Desktop/stylebot_project
/Users/parkheondo/Desktop/stylebot_project/colab-sg2-ada
'stylegan2-ada'에 복제합니다...
remote: Enumerating objects: 71, done.
remote: Total 71 (delta 0), reused 0 (delta 0), pack-reused 71
오브젝트를 받는 중: 100% (71/71), 1.28 MiB | 11.04 MiB/s, 완료.
델타를 알아내는 중: 100% (22/22), 완료.
/Users/parkheondo/Desktop/stylebot_project/colab-sg2-ada/stylegan2-ada


In [4]:
%cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git checkout origin/main -- train.py

/Users/parkheondo/Desktop/stylebot_project/colab-sg2-ada/stylegan2-ada


### 데이터 전처리 과정

In [ ]:
import numpy as np
import scipy.ndimage
import os
import PIL.Image


def image_align(src_file, dst_file, face_landmarks, output_size=256, transform_size=4096, enable_padding=True):
        # Align function from FFHQ dataset pre-processing step
        # https://github.com/NVlabs/ffhq-dataset/blob/master/download_ffhq.py

        lm = np.array(face_landmarks)
        lm_chin          = lm[0  : 17]  # left-right
        lm_eyebrow_left  = lm[17 : 22]  # left-right
        lm_eyebrow_right = lm[22 : 27]  # left-right
        lm_nose          = lm[27 : 31]  # top-down
        lm_nostrils      = lm[31 : 36]  # top-down
        lm_eye_left      = lm[36 : 42]  # left-clockwise
        lm_eye_right     = lm[42 : 48]  # left-clockwise
        lm_mouth_outer   = lm[48 : 60]  # left-clockwise
        lm_mouth_inner   = lm[60 : 68]  # left-clockwise

        # Calculate auxiliary vectors.
        eye_left     = np.mean(lm_eye_left, axis=0)
        eye_right    = np.mean(lm_eye_right, axis=0)
        eye_avg      = (eye_left + eye_right) * 0.5
        eye_to_eye   = eye_right - eye_left
        mouth_left   = lm_mouth_outer[0]
        mouth_right  = lm_mouth_outer[6]
        mouth_avg    = (mouth_left + mouth_right) * 0.5
        eye_to_mouth = mouth_avg - eye_avg

        # Choose oriented crop rectangle.
        x = eye_to_eye - np.flipud(eye_to_mouth) * [-1, 1]
        x /= np.hypot(*x)
        x *= max(np.hypot(*eye_to_eye) * 2.0, np.hypot(*eye_to_mouth) * 1.8)
        y = np.flipud(x) * [-1, 1]
        c = eye_avg + eye_to_mouth * 0.1
        quad = np.stack([c - x - y, c - x + y, c + x + y, c + x - y])
        qsize = np.hypot(*x) * 2

        # Load in-the-wild image.
        if not os.path.isfile(src_file):
            print('\nCannot find source image. Please run "--wilds" before "--align".')
            return
        img = PIL.Image.open(src_file)

        # Shrink.
        shrink = int(np.floor(qsize / output_size * 0.5))
        if shrink > 1:
            rsize = (int(np.rint(float(img.size[0]) / shrink)), int(np.rint(float(img.size[1]) / shrink)))
            img = img.resize(rsize, PIL.Image.ANTIALIAS)
            quad /= shrink
            qsize /= shrink

        # Crop.
        border = max(int(np.rint(qsize * 0.1)), 3)
        crop = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
        crop = (max(crop[0] - border, 0), max(crop[1] - border, 0), min(crop[2] + border, img.size[0]), min(crop[3] + border, img.size[1]))
        if crop[2] - crop[0] < img.size[0] or crop[3] - crop[1] < img.size[1]:
            img = img.crop(crop)
            quad -= crop[0:2]

        # Pad.
        pad = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
        pad = (max(-pad[0] + border, 0), max(-pad[1] + border, 0), max(pad[2] - img.size[0] + border, 0), max(pad[3] - img.size[1] + border, 0))
        if enable_padding and max(pad) > border - 4:
            pad = np.maximum(pad, int(np.rint(qsize * 0.3)))
            img = np.pad(np.float32(img), ((pad[1], pad[3]), (pad[0], pad[2]), (0, 0)), 'reflect')
            h, w, _ = img.shape
            y, x, _ = np.ogrid[:h, :w, :1]
            mask = np.maximum(1.0 - np.minimum(np.float32(x) / pad[0], np.float32(w-1-x) / pad[2]), 1.0 - np.minimum(np.float32(y) / pad[1], np.float32(h-1-y) / pad[3]))
            blur = qsize * 0.02
            img += (scipy.ndimage.gaussian_filter(img, [blur, blur, 0]) - img) * np.clip(mask * 3.0 + 1.0, 0.0, 1.0)
            img += (np.median(img, axis=(0,1)) - img) * np.clip(mask, 0.0, 1.0)
            img = PIL.Image.fromarray(np.uint8(np.clip(np.rint(img), 0, 255)), 'RGB')
            quad += pad[:2]

        # Transform.
        img = img.transform((transform_size, transform_size), PIL.Image.QUAD, (quad + 0.5).flatten(), PIL.Image.BILINEAR)
        if output_size < transform_size:
            img = img.resize((output_size, output_size), PIL.Image.ANTIALIAS)

        # Save aligned image.
        img.save(dst_file, 'PNG')

In [ ]:
import dlib


class LandmarksDetector:
    def __init__(self, predictor_model_path):
        """
        :param predictor_model_path: path to shape_predictor_68_face_landmarks.dat file
        """
        self.detector = dlib.get_frontal_face_detector() # cnn_face_detection_model_v1 also can be used
        self.shape_predictor = dlib.shape_predictor(predictor_model_path)

    def get_landmarks(self, image):
        img = dlib.load_rgb_image(image)
        dets = self.detector(img, 1)

        for detection in dets:
            face_landmarks = [(item.x, item.y) for item in self.shape_predictor(img, detection).parts()]
            yield face_landmarks

In [ ]:
import os
import sys
import bz2
import tensorflow as tf

LANDMARKS_MODEL_URL = 'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2'


def unpack_bz2(src_path):
    data = bz2.BZ2File(src_path).read()
    dst_path = src_path[:-4]
    with open(dst_path, 'wb') as fp:
        fp.write(data)
    return dst_path


if __name__ == "__main__":
    """
    Extracts and aligns all faces from images using DLib and a function from original FFHQ dataset preparation step
    python align_images.py /raw_images /aligned_images
    """

    landmarks_model_path = unpack_bz2(tf.keras.utils.get_file('shape_predictor_68_face_landmarks.dat.bz2',
                                               LANDMARKS_MODEL_URL, cache_subdir='temp'))
    
    RAW_IMAGES_DIR = '전처리 전 데이터의 파일경로' # 전처리 전 데이터의 파일경로
    ALIGNED_IMAGES_DIR = '# 전처리 후 데이터를 받을 파일경로' # 전처리 후 데이터를 받을 파일경로

    landmarks_detector = LandmarksDetector(landmarks_model_path)
    for img_name in [f for f in os.listdir(RAW_IMAGES_DIR) if f[0] not in '._']:
        raw_img_path = os.path.join(RAW_IMAGES_DIR, img_name)
        for i, face_landmarks in enumerate(landmarks_detector.get_landmarks(raw_img_path), start=1):
            face_img_name = '%s_%02d.jpg' % (os.path.splitext(img_name)[0], i)
            aligned_face_path = os.path.join(ALIGNED_IMAGES_DIR, face_img_name)
            os.makedirs(ALIGNED_IMAGES_DIR, exist_ok=True)
            image_align(raw_img_path, aligned_face_path, face_landmarks)

### 데이터셋 만드는 과정

styleGAN2에 원하는 데이터를 학습시키기전에 일단 데이터셋으로 만들어주어야합니다. styleGAN2에 데이터셋을 만드는 기능이 있어서 바로 활용했습니다.

In [ ]:
#update this to the path to your image folder
dataset_path = "데이터셋의 경로를 넣어주세요"
#give your dataset a name
dataset_name = "데이터셋의 이름을 입력해주세요"

#you don't need to edit anything here
!python dataset_tool.py create_from_images /content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/datasets/{dataset_name} {dataset_path}

### 데이터 학습 과정

첫번째 셀을 실행시키면 train.py를 실행시키는데 parameter를 볼 수 있습니다. 필수로 필요한 파라미터는 outdir, data가 있습니다.

In [5]:
!python train.py --help

Traceback (most recent call last):
  File "train.py", line 18, in <module>
    import dnnlib.tflib as tflib
  File "/Users/parkheondo/Desktop/stylebot_project/colab-sg2-ada/stylegan2-ada/dnnlib/tflib/__init__.py", line 9, in <module>
    from . import autosummary
  File "/Users/parkheondo/Desktop/stylebot_project/colab-sg2-ada/stylegan2-ada/dnnlib/tflib/autosummary.py", line 32, in <module>
    from . import tfutil
  File "/Users/parkheondo/Desktop/stylebot_project/colab-sg2-ada/stylegan2-ada/dnnlib/tflib/tfutil.py", line 18, in <module>
    import tensorflow.contrib   # requires TensorFlow 1.x!
ModuleNotFoundError: No module named 'tensorflow.contrib'


In [ ]:
!python train.py \
 --outdir='/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results' \
 --snap=4 \
 --data='데이터셋의 경로를 입력해주세요' \
 --mirror=true \
 --metrics=none \
 --cfg=auto \
 --resume='사전학습모델의 경로를 입력해주세요' \
 --kimg=300 \
 --freezed=10 \

--outdir에는 학습결과를 받을 파일경로를 입력해줍니다.

--data에는 dataset이 들어있는 경로를 입력해줍니다.

--resume의 경우 사전학습모델이 있다면 입력해줍니다.

--kimg의 경우는 학습시킬 이미지 수를 입력해줍니다.

--freezed는 판별자를 얼마나 얼릴지 지정해주는 변수입니다. 과적합이 일어나면 판별자층을 어느정도 얼려주어서 기존에 학습한 가중치를 최대한 활용하는 것이 <br>좋습니다.

--snap의 경우 snapshot을 어느주기로 남겨줄지 정해주는 겁니다. 예를 들어 4로 설정해두면 이미지 4개를 학습할 때 마다 학습모델과 결과를 파일로 남겨줍니다.

### Image generate

In [ ]:
pip install opensimplex

In [ ]:
!python generate.py generate-images --outdir=out2 --trunc=1 --seeds=401-800 \
      --network='./results/00048-short-mirror-auto1-kimg300-resumecustom-freezed10/network-snapshot-000300.pkl'

generate를 실행하려면 opensimplex를 설치해주어야합니다. 

--network는 어떤 모델로 훈련시킬 지 지정해주는 변수입니다. 모델이 있는 파일경로를 넣어주면 됩니다.

--outdir는 생성된 이미지를 어디로 받을 지 경로를 지정해줍니다.

--seeds는 시드를 지정해주는 변수입니다. 랜덤으로 넣어주셔도 되고 지정해서 넣어주셔도 됩니다.